In [1]:
import numpy as np
import numpy.linalg as linalg

def minimum_volume_enclosing_ellipsoid(points, tol=0.01):
    """
    Computes the minimum volume enclosing ellipsoid of a set of points in n-dimensional space.

    Args:
        points: A d-by-N numpy array of N points in R^d.
        tol: The tolerance for convergence of the iterative algorithm (default 0.01).

    Returns:
        A tuple containing the A matrix and center c such that (x-c)^T * A * (x-c) = 1 defines the ellipsoid.
    """

    d, N = points.shape

    # Embed the points into a higher-dimensional space by adding an extra dimension and setting it to 1.
    Q = np.zeros((d+1, N))
    Q[:d, :] = points
    Q[d, :] = np.ones(N)

    # Initialize the weights to be uniform.
    u = (1.0/N) * np.ones(N)

    # Iterate until convergence.
    err = 1
    while err > tol:
        # Compute the covariance matrix of the weighted points.
        X = Q @ np.diag(u) @ Q.T

        # try:
        # Compute the eigenvalues and eigenvectors of the covariance matrix.
        eigvals, eigvecs = linalg.eigh(X)

        # Find the largest eigenvalue and its corresponding eigenvector.
        max_eigval = np.max(eigvals)
        # max_eigvec = eigvecs[:, np.argmax(eigvals)]

        # Update the weights.
        step_size = (max_eigval - d - 1) / ((d+1) * (max_eigval - 1))
        new_u = (1 - step_size) * u
        new_u[np.argmax(eigvals)] += step_size

        # Compute the norm of the difference between the new and old weights.
        err = linalg.norm(new_u - u)
        u = new_u
        # except:
            # If the covariance matrix is not invertible, return None.
            # return None

    # Compute the center of the ellipsoid.
    c = points @ u

    return c,u

    # try:
    #     # Compute the A matrix of the ellipsoid.
    #     A = (1.0/d) * linalg.inv(points @ np.diag(u) @ points.T - np.outer(c, c))
    # except:
    #     # If the matrix is not invertible, return None.
    #     return None

    # return (A, c)


In [2]:
import pandas as pd
from scipy.linalg import hankel
df = pd.read_csv('~/data/final_final.csv')
for sensor in range(20):
    xmeas = df.iloc[:,sensor].values
    X_train_data = xmeas[:2000] # For training 
    N = len(X_train_data)
    L = N // 2
    X_train = hankel(X_train_data[:L],X_train_data[L-1:])
    # X_train =  X_train[:,:-1]
    # print(np.linalg.det(X_train))
    X_theta = xmeas[2000:8000] # For threshold calculation
    X_test = xmeas[8000:] # For detection
    c,u = minimum_volume_enclosing_ellipsoid(X_train)
    # print(c)
    break

In [3]:
from models import Hankel,Rank,Cluster,Meepc
meepc = Meepc.MEEPC()
weights,ce = meepc.fit(X_train)

/home/manu/meepc/models/Meepc.py:34: RuntimeWarning: divide by zero encountered in scalar divide
  ans += (1/(u + s_i_j/Z[i][j]))
